In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)

processor = AutoProcessor.from_pretrained(model_id)

# Save Models
model.save_pretrained('./models/openai-whisper-large-v3')

processor.save_pretrained('./models/openai-whisper-large-v3')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# model.to(device)

# processor = AutoProcessor.from_pretrained(model_id)

# pipe = pipeline(
#     "automatic-speech-recognition",
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     max_new_tokens=128,
#     chunk_length_s=30,
#     batch_size=16,
#     return_timestamps=True,
#     torch_dtype=torch_dtype,
#     device=device,
# )

# # save model
# import pickle
# f = open('./model/whisper.pkl', 'wb')
# pickle.dump(model, f)
# f.close()

# # save processor
# f = open('./model/processor.pkl', 'wb')
# pickle.dump(processor, f)
# f.close()

# # save pipeline
# f = open('./model/pipeline.pkl', 'wb')
# pickle.dump(pipe, f)
# f.close()

In [7]:
import pickle
f = open('./whisper-api/models/whisper.pkl', 'rb')
model = pickle.load(f)
f.close()

# save processor
f = open('./whisper-api/models/processor.pkl', 'rb')
processor = pickle.load(f)
f.close()

# save pipeline
f = open('./whisper-api/models/pipeline.pkl', 'rb')
pipe = pickle.load(f)
f.close()

In [13]:
# !pip install ffmpeg

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# result = pipe(sample)

result = pipe(outfile_path, generate_kwargs={"task": "translate"})
print(result["text"])

 You need about eight ounces or 240 milliliters of fluid per hour for the first 10 hours of your day after waking. Now, why did I say for the first 10 hours? Well, it turns out that the filtration of fluids from your body, which is accomplished of course, by your kidneys and by way of your bladder and the excretion of fluid out your urethra, so-called urination, is strongly, strongly circadian dependent, meaning the cells of your body and the excretion of fluid out urethra, so-called urination, is strongly, strongly circadian dependent, meaning the cells of your kidney and the cells even of your gut, in fact, all the cells of your body, but especially the cells of your kidney, which filter the fluid that comes into your body and that makes certain hormones like vasopressin, which is anti-diuretic hormone, all of that functioning of the kidney is under strong regulation by so-called circadian clock genes. Circadian clock genes are genes that are expressed in every cell, but that in cert

In [14]:
result = pipe(infile_path, generate_kwargs={"task": "translate"})
print(result["text"])

 You need about eight ounces or 240 milliliters of fluid per hour for the first 10 hours of your day after waking. Now, why did I say for the first 10 hours? Well, it turns out that the filtration of fluids from your body, which is accomplished of course, by your kidneys and by way of your bladder and the excretion of fluid out your urethra, so-called urination, is strongly, strongly circadian dependent, meaning the cells of your body and the excretion of fluid out urethra, so-called urination, is strongly, strongly circadian dependent, meaning the cells of your kidney and the cells even of your gut, in fact, all the cells of your body, but especially the cells of your kidney, which filter the fluid that comes into your body and that makes certain hormones like vasopressin, which is anti-diuretic hormone, all of that functioning of the kidney is under strong regulation by so-called circadian clock genes. Circadian clock genes are genes that are expressed in every cell, but that in cert

## Seq2Seq Summarization using T5
- https://huggingface.co/t5-small

In [18]:
import json

start_one, end_one = 54.67, 99.33
start_two, end_two = 270.67, 1297.33

raw_text_path = './api/db/7e7fe030-02c7-4481-b1de-b2b607847d7f/results.json'
result = json.load(open(raw_text_path, 'r')).get('result', {})

text = 'summarize: '

for chunk in result.get('chunks', []):
    timestamp = chunk['timestamp']
    if timestamp[0] > start_one and timestamp[1] < end_one:
        text += chunk['text']
    if timestamp[0] > start_two and timestamp[1] < end_two:
        text += chunk['text']
    
text


"summarize:  This explosion left all Puerto Rico dark in April. In August, one of the country's main hospitals was almost 20 hours without electricity. All this after the distribution of energy in Puerto Rico went to the hands of a private company. The government granted a multimillion-dollar contract to a company in the United States and Canada called Luma Energy, and they promised the people that the service would be better. But according to the government's metrics, now the shutdowns are longer. The result is students who do their assignments in the dark, in damaged beings and people who cannot connect their medical team at home. At the same time, the people Rican people have seen seven consecutive increases in the electricity bill, while the executives of Luma are charging salaries never seen before in the Puerto R Rico electric corporation. With the famous letter-day card and you have to go. And now where do I go? And if what they charge me there where I can, I won't give up with 

In [26]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs.shape

Token indices sequence length is longer than the specified maximum sequence length for this model (4119 > 512). Running this sequence through the model will result in indexing errors


torch.Size([1, 4119])

In [47]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
outputs = model.generate(inputs, max_new_tokens=1500, min_length=1, do_sample=True)

In [48]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'mayor was born in dorado, and told a friend of her mother to come to town. the company is part of a law banning property at the back of the wall. it wants other people to benefit financially from it, a person from the north. meanwhile, a man who bought land on the beach can use it to live there.'